**Creating a 3D array of size (2, 9, 5)**
```python
array_3d = np.array([
    [
        [1, 2, 3, 4, 5],
        [6, 7, 8, 9, 10],
        [41, 42, 43, 44, 45]
    ],
    [
        [46, 47, 48, 49, 50],
        [81, 82, 83, 84, 85],
        [86, 87, 88, 89, 90]
    ]
])
```

**Get flatten array from index [:,:,1]**
```python
array_3d[:,:,1].flatten() 
```

[ 2  7 42 47 82 87]

=**Get flatten array from index [:,1,:]**
```python
array_3d[:,1,:].flatten()
```

[ 6  7  8  9 10 81 82 83 84 85]

**Get flatten array from index [1,:,:]**
```python
array_3d[1,:,:].flatten()
```

[46 47 48 49 50 81 82 83 84 85 86 87 88 89 90]


In [1]:
import {InferenceSession, Tensor, env} from 'onnxruntime-web';
const ndarray = require('ndarray')
const ops = require('ndarray-ops')
const fs = require('fs')
import Jimp from "jimp";
import { Niivue, NVImage } from "@niivue/niivue";
import { cv } from "opencv-wasm";


In [2]:
// in kernel, The configuration runs using node, and not in the browser,
// which is why window is not defined
// we need to define it to make the code work

// global.window = { navigator: { userAgent: 'node.js' } };

import crypto from "node:crypto";
global.crypto ??= crypto;
const volume = await NVImage.loadFromUrl({url:"https://niivue.github.io/niivue-demo-images/mni152.nii.gz"})
console.log(volume.img)

// import fs from 'fs';

// const writeStream = fs.createWriteStream('file.txt');
// const pathName = writeStream.path;
  
// // write each value of the array on the file breaking line
// volume.img.forEach(value => writeStream.write(`${value}\n`));

// // the finish event is emitted when all data has been flushed from the stream
// writeStream.on('finish', () => {
//    console.log(`wrote all the array data to file ${pathName}`);
// });

// // handle the errors on the write process
// writeStream.on('error', (err) => {
//     console.error(`There is an error writing the file ${pathName} => ${err}`)
// });

// // close the stream
// writeStream.end();

Uint8Array(11393280) [
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
  0, 0, 0, 0,
  ... 11393180 more items
]


In [3]:
function getMax(arr) {
    let len = arr.length;
    let max = -Infinity;

    while (len--) {
        max = arr[len] > max ? arr[len] : max;
    }
    return max;
}

function getMin(arr) {
    let len = arr.length;
    let min = Infinity;

    while (len--) {
        min = arr[len] < min ? arr[len] : min;
    }
    return min;
}

In [4]:
function normalizeArray(
    array: Float32Array | Uint8Array,
    max: number
  ): Float32Array {
    let normalizedArray = new Float32Array(array.length);
    for (let i = 0; i < array.length; i++) {
      normalizedArray[i] = array[i] / max;
    }
    return normalizedArray;
  }
  
  function stackSliceToRGB(
    buffer: Float32Array | Uint8Array,
  ): Float32Array {
    let bufferLength = buffer.length,
      result = new Float32Array(bufferLength * 3);
  
    for (let i = 0; i < bufferLength; i++) {
      result[3 * i] = buffer[i];
      result[3 * i + 1] = buffer[i];
      result[3 * i + 2] = buffer[i];
    }
    return result;
  }
  
  

In [5]:
const preprocessVolume = (image) => {
    try {
      const imageRAS = image.img2RAS();
      // console.log("image.calMax, image.calMin ", image.global_max, image.global_min);
      const maxVal = getMax(imageRAS);
      const normalizedArray = normalizeArray(imageRAS, maxVal);
      console.log("normalizedArray ", normalizedArray.reduce((a, b) => a + b, 0));
      return normalizedArray;
    } catch (e) {
      console.log(`failed to preprocess volume: ${e}. `);
      throw Error(`failed to preprocess volume: ${e}. `);
    }
  }

const preprocess = (slice2D: Float32Array, sliceId: number, width: number, height: number): Float32Array => {
    try {
      // console.log("imageArray ", slice2D);
      let image3Channels: Float32Array;
      const sliceArray = slice2D.slice(
        width * height * sliceId,
        width * height * (sliceId + 1),
      );
      console.log("sliceArray ", sliceArray.reduce((a, b) => a + b, 0));
  
      image3Channels = stackSliceToRGB(sliceArray);
      console.log("image3Channels ", image3Channels, image3Channels.reduce((a, b) => a + b, 0));
      return image3Channels;
    } catch (e) {
      console.log(`failed to inference ONNX model: ${e}. `);
      throw Error(`failed to inference ONNX model: ${e}. `);
    }
};


In [6]:
function imagedataToImage(imagedata: Float32Array | Uint8Array, dims: number[]) {
    let inputImage;
    // if (dims[0]*dims[1] !== imagedata.length) {
    //     throw new Error("Image data size does not match the dimensions");
    // }

      inputImage = new Uint8Array(dims[0]*dims[1]*3);
      for (let i=0; i<imagedata.length; i++) {
        inputImage[i] = imagedata[i]*255;
    }

    let dst = cv.matFromArray(dims[0], dims[1], cv.CV_8UC3, inputImage);
    console.log(dst.data.slice(600, 610));
    console.log(dst);
    new Jimp({
        width: dims[0],
        height: dims[1],
        data: Buffer.from(dst.data)
    }).write('./niivue.png');
}


In [7]:
function nvVolumetoSlice(volume, dims, fov, index): any {
    let sliceArray;
    switch (fov) {
        // sagittal dimsRAS[1], coronal dimsRAS[2], axial dimsRAS[3]
        
        // 1. if axial, slice the array from volume
        case "axial":
            sliceArray = volume.slice(
                dims[0] * dims[1] * index,
                dims[0] * dims[1] * (index + 1),
            );
            console.log("axial sliceArray",dims[1], dims[2], index, sliceArray.length);
        // 2. if coronal, 
        case "coronal":
        // 3. if sagittal,
        case "sagittal":
    }
    return sliceArray;
    
}

function nvSliceToTensor(nvSlice, dims): any {
    console.log("nvSlice", nvSlice.length);
    // 1. preprocess
    let image3Channels = stackSliceToRGB(nvSlice);
    // 2. convert to float32
    console.log("image3Channels", image3Channels.length);
    // 3. tensor
    const inputTensor = new Tensor("float32", image3Channels, dims);
    return inputTensor;
}

function imageDataToTensor(data, dims, max): any {
    // 1a. Extract the R, G, and B channels from the data to form a 3D int array
    const [R, G, B]: number[][] = [[], [], []];
    for (let i = 0; i < data.length; i += 3) {
      R.push(data[i]);
      G.push(data[i + 1]);
      B.push(data[i + 2]);
      // 2. skip data[i + 3] thus filtering out the alpha channel
    }
    // 1b. concatenate RGB ~= transpose [224, 224, 3] -> [3, 224, 224]
    const transposedData = R.concat(G).concat(B);
  
    // 3. convert to float32
    let i, l = transposedData.length; // length, we need this for the loop
    const float32Data = new Float32Array(dims[3]*dims[1]*dims[2]); // create the Float32Array for output
    for (i = 0; i < l; i++) {
      float32Data[i] = transposedData[i] ; // divide by max value to convert to float
    }
  
    const inputTensor = new Tensor("float32", float32Data, dims);
    return inputTensor;
  }

function arrayToTensor(array, dims): any {
    let float32Data = new Float32Array(array.length*3);
    for (let i = 0; i < array.length; i++) {
        float32Data[i] = array[i];
        float32Data[i + array.length] = array[i];
        float32Data[i + 2 * array.length] = array[i];
    }
    const inputTensor = new Tensor("float32", float32Data, dims);
    return inputTensor;
}

In [8]:
const dims = [volume.dimsRAS[1], volume.dimsRAS[2], volume.dimsRAS[3]];
let nvVolume = preprocessVolume(volume);
let nvSlice = nvVolumetoSlice(nvVolume, dims, "axial", 90);
let inputTensor = nvSliceToTensor(nvSlice, [1, 3, dims[0], dims[1]]);
const maxVal = getMax(volume.img2RAS());
let transposedTensor = imageDataToTensor(inputTensor.data, [1, 3, dims[1], dims[0]], maxVal);

normalizedArray  3269611.783972291
axial sliceArray 256 215 90 52992
nvSlice 52992


image3Channels 158976


In [11]:
inputTensor.data.slice(16000,16007)

Float32Array(7) [
  0.4246031641960144,
  0.4246031641960144,
  0.2936508059501648,
  0.2936508059501648,
  0.2936508059501648,
  0.1626984179019928,
  0.1626984179019928
]


In [80]:
// const dims = [volume.dimsRAS[1], volume.dimsRAS[2], volume.dimsRAS[3]];
// // let nvVolume = preprocessVolume(volume);
// let nvSlice = nvVolumetoSlice(volume.img2RAS(), dims, "axial", 90);
// // let inputImage = nvSliceToImage(nvSlice);
// const maxVal = getMax(volume.img2RAS());
// let inputTensor = arrayToTensor(nvSlice, [1, 3, dims[0], dims[1]], maxVal);

axial sliceArray 256 215 90 52992


In [9]:
imagedataToImage(inputTensor.data, [dims[0], dims[1]]);

Uint8Array(10) [
  0, 0, 0, 0, 0,
  0, 0, 0, 0, 0
]
Mat {}


In [30]:
console.log(transposedTensor.data.reduce((a,b) => a+b,0), transposedTensor.dims, getMax(transposedTensor.data), getMin(transposedTensor.data));

console.log(volume.img2RAS().reduce((a,b) => a+b, 0), getMax(volume.img), getMin(volume.img), volume.global_max, volume.global_min);

77127.94049503142 [ 1, 3, 256, 207 ] 0.932539701461792 0
823942169 252 0 80 40


In [24]:
console.log(inputTensor.data.reduce((a,b) => a+b,0), inputTensor.dims, getMax(inputTensor.data), getMin(inputTensor.data));

console.log(volume.img2RAS().reduce((a,b) => a+b, 0), getMax(volume.img), getMin(volume.img), volume.global_max, volume.global_min);

77127.94049503142 [ 1, 3, 256, 207 ] 0.932539701461792 0
823942169 252 0 80 40


In [10]:
global.self = global;
env.wasm.numThreads = 1;

const encoderSession = await InferenceSession.create('../public/model/efficient_sam_vitt_encoder.onnx');
const decoderSession = await InferenceSession.create('../public/model/efficient_sam_vitt_decoder.onnx');


In [11]:
async function runEncoder(model, preprocessedData): Promise<[Tensor, number]> {
    const start = new Date();
    try {
      const feeds: Record<string, Tensor> = {};
      feeds[model.inputNames[0]] = preprocessedData;
      const outputData = await model.run(feeds);
      const end = new Date();
      const inferenceTime = (end.getTime() - start.getTime());
      const output = outputData[model.outputNames[0]];
      return [output, inferenceTime];
    } catch (e) {
      console.error(e);
      throw new Error();
    }
  }

async function runDecoder(model, embedding, inputPoints, inputLabels, oriSize): Promise<[Tensor, number]> {
  const decoderStart = new Date();
  try {
    const decoderFeeds: Record<string, Tensor> = {};
    console.log("model.inputNames", model.inputNames)
    decoderFeeds[model.inputNames[0]] = embedding;
    decoderFeeds[model.inputNames[1]] = inputPoints;
    decoderFeeds[model.inputNames[2]] = inputLabels;
    decoderFeeds[model.inputNames[3]] = oriSize;

    // feeds[model.inputNames[1]] = new Tensor("float32", inputPoints, [1, 1, inputPoints.length/2, 2]);
    // feeds[model.inputNames[2]] = new Tensor("float32", new Array(inputPoints.length/2).fill(1), inputPoints.length/2);
    // feeds[model.inputNames[3]] = new Tensor("int64", oriSize);
    const outputData = await model.run(decoderFeeds);
    const decoderEnd = new Date();
    const decoderInferenceTime = (decoderEnd.getTime() - decoderStart.getTime());
    const decoderOutput = outputData[model.outputNames[0]];
    return [decoderOutput, decoderInferenceTime];
  } catch (e) {
    console.error(e);
    throw new Error();
  }
}

In [26]:
const [embedding, encoderTime] =  await runEncoder(encoderSession, transposedTensor);
var encoderOutput = embedding.data;
var inferenceTime = encoderTime;
// var results = softmax(Array.prototype.slice.call(output))

In [27]:
console.log((encoderOutput as Float32Array).reduce((a,b) => a + b, 0), inferenceTime);

2118.37974857889 5364


In [28]:
let points = new Float32Array([134, 150])
let labels = new Float32Array(points.length/2).fill(1)
let pointsTensor: Tensor = new Tensor("float32", points, [1, 1, 1, 2]);
let labelsTensor: Tensor = new Tensor("float32", labels, [1,1,labels.length]);
let originalSize: Tensor = new Tensor("int64", [dims[1], dims[0]], [2]);
const [mask, decoderTime] = await runDecoder(decoderSession, embedding, pointsTensor, labelsTensor, originalSize);
var decoderOutput = mask.data;
var inferenceTime = decoderTime;

model.inputNames [
  'image_embeddings',
  'batched_point_coords',
  'batched_point_labels',
  'orig_im_size'
]


In [15]:
function transposeChannel(data, dims) {
    let transposedData = new Float32Array(dims[3]*dims[4]*dims[2]);
    const channelLength = dims[3]*dims[4];
    for (let i = 0; i < data.length; i+=3) {
        transposedData[i] = data[i];
        transposedData[i+1] = data[i+channelLength];
        transposedData[i+2] = data[i+2*channelLength];
    }
    return transposedData;
}

In [59]:
let a = [0,2,4,5]
mask.dims.slice(3,)

[ 256, 207 ]


In [42]:
// let transposedMask = imageDataToTensor(decoderOutput, mask.dims.slice(2,), 1);

In [89]:
let maskImage = new Uint8Array(dims[0]*dims[1]);
let decoderOutputSlice = (decoderOutput as Float32Array).slice(0,dims[0]*dims[1]);
for (let i=0; i<decoderOutputSlice.length; i++) {
    if (decoderOutputSlice[i] >= 0) {
        maskImage[i] = 1;
        // maskImage[i+1] = 1;
        // maskImage[i+2] = 1;
    } else {
        maskImage[i] = 0;
        // maskImage[i+1] = 0;
        // maskImage[i+2] = 0;
    }
}
// let transposedMask = transposeChannel(maskImage, mask.dims)


In [79]:
decoderOutputSlice

Float32Array(52992) [
  -3.8637492656707764,  -3.991184711456299,  -3.871520757675171,
  -3.9694225788116455, -3.9836342334747314,  -4.037299633026123,
  -3.7964789867401123, -3.8899168968200684,  -4.081902503967285,
  -3.6294565200805664, -3.7999801635742188, -3.9474992752075195,
  -3.5137012004852295, -3.7479207515716553,  -3.833216667175293,
  -3.6118245124816895, -3.6930220127105713, -3.7316269874572754,
  -3.7313129901885986,  -3.580113172531128,  -3.673612117767334,
    -3.85579514503479,  -3.466813564300537,  -3.650151014328003,
  -3.8262174129486084, -3.3521599769592285, -3.6383109092712402,
   -3.736264228820801, -3.4498677253723145, -3.6410231590270996,
   -3.667262077331543, -3.5994133949279785, -3.5492658615112305,
  -3.6287851333618164, -3.7613775730133057,  -3.454685688018799,
  -3.6444153785705566,  -3.826319932937622,  -3.360084056854248,
  -3.6582911014556885, -3.7635347843170166,  -3.414520263671875,
  -3.6786434650421143, -3.7053258419036865,   -3.56730318069458,
   

In [86]:
maskImage.reduce((a,b) => a+b, 0)
// getMax(maskImage)

maskImage.length

158976


In [90]:
let stack_mask = stackSliceToRGB(maskImage)

In [91]:
imagedataToImage((stack_mask), [dims[0], dims[1]]);

Uint8Array(10) [
  0, 0, 0, 0, 0,
  0, 0, 0, 0, 0
]
Mat {}


In [57]:
function sigmoid(data) {
    return data.map(x => 1 / (1 + Math.exp(-x)))
}

function softmax(data) { 
    return data.map(x => Math.exp(x) / (data.map(y => Math.exp(y))).reduce((a,b) => a+b)) 
}


-1811852.2913827675
